# Data Analysis

## Solutions object
Obtain the Hidden Sector from: https://zenodo.org/record/5526707#.YzwIANJBzeQ and add the missing the column `Hidden Sector` of the following data scheme

![img](./squeme.svg)

### Example of entry

In [114]:
from IPython.display import JSON 
import pandas as pd

In [ ]:
JSON({'A':1})

<IPython.core.display.JSON object>

In [ ]:
JSON(([{'l':[1,3],'k':[2],
       'solution':[1,2,-3,4,5],'gcd':1,'n':5,
      'hidden':[{'ψ':[(1,5),(2,4),(-3,)],
                 'S':6}
               ]
             }] 
            )
    )


<IPython.core.display.JSON object>

In [1]:
import pandas as pd

In [4]:
pd.DataFrame([{'l':[1,3],'k':[2],
       'solution':[1,2,-3,4,5],'gcd':1,'n':5,
      'hidden':[{'ψ':[(1,5),(2,4),(-3,)],
                 'S':6}
               ]
             }] 
            )

,l,k,solution,gcd,n,hidden
0,"[1, 3]",[2],"[1, 2, -3, 4, 5]",1,5,"[{'ψ': [(1, 5), (2, 4), (-3,)], 'S': 6}]"


### Read the full dataset

In [115]:
df=pd.read_json('https://zenodo.org/record/5526707/files/solutions.json?download=1')

In [2]:
df=pd.read_json('solutions.json.gz')

In [140]:
sorted?

Signature: sorted(iterable, /, *, key=None, reverse=False)
Docstring:
Return a new list containing all items from the iterable in ascending order.

A custom key function can be supplied to customize the sort order, and the
reverse flag can be set to request the result in descending order.
Type:      builtin_function_or_method


In [142]:
sorted([-2,1,-8,5],key=abs)

[1, -2, 5, -8]

In [139]:
df['solution'].str[-1]

0          9
1         10
2         20
3         22
4         25
          ..
390069    32
390070    32
390071   -32
390072    32
390073    32
Name: solution, Length: 390074, dtype: int64

In [116]:
df['zmax']=df['solution'].str[-1].abs()
df=df.sort_values(['n','zmax']).reset_index(drop=True)

#How many solutions with n=12 and z_max <= 20?
df[ (df['n']==12) & (df['zmax']<=20) ].shape

(16862, 6)

### Add the new column with the list of hidden sectors

After refactoring...

In [120]:
import itertools
import math as mh

def remove_double_intersection_pairs(msvp,l):
    for p in msvp.copy():
        tmp=msvp.copy()
        tmp.remove(p)
        fltmp=set([x for sublist in tmp for x in sublist]) 
        if l.count(p[0])==1 and l.count(p[1])==1 and set(p).issubset(fltmp):
            msvp=tmp.copy()
    return msvp
assert len(remove_double_intersection_pairs([(-10, 11), (-12, 13), (11, -12)],
                                             [-10, 11, -12, 13]))==2
assert len(remove_double_intersection_pairs([(4,4),(4,-12)],[4,-12]))==1

def get_real_massless(l,msvp):
    lp=l.copy()
    for x,y in msvp:
        #print(lp,msvp,x,y)
        if lp and x in lp and y in lp:
            minxy=min(lp.count(x),lp.count(y))
            if x!=y:
                [(lp.remove(x),lp.remove(y)) for ii in range(minxy)]
            else:
                [ lp.remove(x) for ii in range(lp.count(x)) ] # abs(2*x)==s
        else: # not lp
            break
    return lp

def get_hidden_sector(l):
    '''
    Extract massive pairs without massles fermions from l
        [1, 3, 4, 6, -7, -8, -13, 14] → [[3,4],[6,-13],[1,6],[-7,14],[1,-8]]
        #Remove [1,6] because is already in the pairs list
        [1, 3, 4, 6, -7, -8, -13, 14] → [[3,4],[6,-13],[-7,14],[1,-8]]
        [1, 6, -7, -8, -13, 14] → [[6,-13],[-7,14],[1,-8]]
        [1, -7, -8, 14] → [[-7,14],[1,-8]]
        [1, -8] → [[1,-8]]
        [] → []    
    '''
    sltns=[]
    ps = list(set(itertools.combinations_with_replacement(l,2)))
    Ss = set(abs(sum(x)) for x in ps)
    for s in Ss:
        msv = [p for p in ps if abs(sum(p))==s]
        lmsv=[x for sublist in msv for x in sublist]
        massives=set(lmsv)        
        massless=set(l).difference(massives)
        if not massless:
            # ========= Quality check =======================
            # Check that all the elements of l can be removed
            for INTERSECTIONS in [False,True]:
                msvp=msv.copy()
                lp=l.copy()
                if INTERSECTIONS:
                    msvp = remove_double_intersection_pairs(msvp,l)
                for msvp in itertools.permutations(msvp.copy()):
                    msvp=list(msvp)
                    lp=get_real_massless(l,msvp)
                    if not lp:
                        break #found real massless
                if not lp: # double break for INTERSECTIONS
                    break # found real massless
                    
            if lp:
                continue # Not a hidden sector
            # =================================
            sltn={}
            sltn['S']=s
            #Save storage
            sltn['ψ']=msv
            sltns.append(sltn)
    return sltns

assert get_hidden_sector([1,2,-3,4,5])[0].get('S')==6
assert get_hidden_sector([1,2,-3,-3,4,5])[0].get('S')==6
assert get_hidden_sector([1,1,2,-3,-3,4,5,5])[0].get('S')==6
#aψ1ψ1+bψ1ψ2+cψ1,ψ3 → https://www.wolframalpha.com/input?i=Rank%20{{a,b,c},{b,0,0},{c,0,0}}
assert get_hidden_sector([1,2,-3,4,5,9,9])==[]
get_hidden_sector([1,1,2,-3,4,5])
#aψ1ψ2+bψ1ψ3 → https://www.wolframalpha.com/input?i=rank+{{0,a,b},{a,0,0},{b,0,0}}
assert get_hidden_sector([1,1,2,-3,4,5])==[]
assert get_hidden_sector([1,1,2,-3,-3,4,5])==[]
assert get_hidden_sector([1,2,-3,4,5,8])==[]
assert get_hidden_sector([1,2,-3,4,5,8,8])==[]
assert get_hidden_sector([])==[]
assert get_hidden_sector([1, 1, 1, 1, 1, -2, -2, -2, -2, 3])==[]
assert get_hidden_sector([1, 2, 2, 2, -3, -5, -6, 7])[0].get('S')==4
assert get_hidden_sector( [1, 2, 2, 4, -5, -5, -7, 8] )[0].get('S')==3
assert get_hidden_sector([2, -3, -4, 5, -6, 7, 7, -8])[0].get('S')==1 # Ana test
assert get_hidden_sector([3, 5, -8, 9, -10, -14, 15])==[] # Dirac triplet [-10,5,15] (s=5)
assert get_hidden_sector([3, 5, -8, 9, -10, -14, 15, 20,-30, 35])==[]
assert get_hidden_sector([1, 3, 3, 3, -5, -7, -7, 9])[0].get('S')==4 #order of pairs matters
assert get_hidden_sector([7, -8, -10, 11, -12, 13, 14, -15])[0].get('S')==1 #double intersection pair
assert get_hidden_sector([4,-12])[0].get('S')==8

In [118]:
%%time
df['hidden']=df['solution'].apply(get_hidden_sector)

CPU times: user 1min 45s, sys: 0 ns, total: 1min 45s
Wall time: 1min 45s


In [127]:
hs=df[df['hidden'].apply(len)>0].reset_index(drop=True)#.iloc[0].to_dict()

In [128]:
hs.shape #9466 solutions with hidden sector

(9466, 7)

In [129]:
hs[:1]

,l,k,solution,gcd,n,zmax,hidden
0,"[-1, -5, -3]","[-6, -4, -7]","[1, 1, 2, 3, -4, -4, -5, 6]",1,8,6,"[{'S': 2, 'ψ': [(3, -5), (2, -4), (-4, 6), (1, 1)]}]"


Ana

In [181]:
def get_hidden_sector(l):

    com = list(itertools.combinations_with_replacement(l, 2))
    suma = set([abs(sum(i)) for i in com])

    final = []

    for s in suma:
        cond = [i for i in com if abs(sum(i)) == s]
        aplanar = [item for sublist in cond for item in sublist]
        resta = set(l).difference(set(aplanar))

        if not resta:
            l_ = l.copy()
            cond_ = cond
            N = len(l)

            combs = []

            equal = [par for par in cond_ if par[0] == par[1]]
            ind_eq = [cond.index(eq) for eq in equal]
            
            for ind in ind_eq:
                cond_.pop(ind)

            for par in cond_:
                try:
                    for val in par:
                        l_[l_.index(val)] = 0
                    N = N - 2
                    combs.append(par)
                except:
                    pass
                    # print("Discharge pair", par)
            
            for par in equal:
                ind = []
                for val in par:
                    try:
                        ind.append(l_.index(val))
                    except:
                        pass
                for ind_ in ind:
                    l_[ind_] = 0

                if len(set(ind)) == 1:
                    N = N - 1
                elif len(set(ind)) != 1:
                    N = N - len(set(ind))

                combs.append(par)

            if N == 0:
                final.append({"S": s, "ψ": combs})

            del l_, cond_

    return final

assert get_hidden_sector([1,2,-3,4,5])[0].get('S')==6
assert get_hidden_sector([1,2,-3,-3,4,5])[0].get('S')==6
assert get_hidden_sector([1,1,2,-3,-3,4,5,5])[0].get('S')==6
#aψ1ψ1+bψ1ψ2+cψ1,ψ3 → https://www.wolframalpha.com/input?i=Rank%20{{a,b,c},{b,0,0},{c,0,0}}
assert get_hidden_sector([1,2,-3,4,5,9,9])==[]
get_hidden_sector([1,1,2,-3,4,5])
#aψ1ψ2+bψ1ψ3 → https://www.wolframalpha.com/input?i=rank+{{0,a,b},{a,0,0},{b,0,0}}
assert get_hidden_sector([1,1,2,-3,4,5])==[]
assert get_hidden_sector([1,1,2,-3,-3,4,5])==[]
assert get_hidden_sector([1,2,-3,4,5,8])==[]
assert get_hidden_sector([1,2,-3,4,5,8,8])==[]
assert get_hidden_sector([])==[]
assert get_hidden_sector([1, 1, 1, 1, 1, -2, -2, -2, -2, 3])==[]
assert get_hidden_sector([1, 2, 2, 2, -3, -5, -6, 7])[0].get('S')==4
assert get_hidden_sector( [1, 2, 2, 4, -5, -5, -7, 8] )[0].get('S')==3
# Ana test
assert get_hidden_sector([2, -3, -4, 5, -6, 7, 7, -8])[0].get('S')==1
assert get_hidden_sector([3, 5, -8, 9, -10, -14, 15])==[] # Dirac triplet [-10,5,15] (s=5)
assert get_hidden_sector([3, 5, -8, 9, -10, -14, 15, 20,-30, 35])==[]
assert get_hidden_sector([1, 3, 3, 3, -5, -7, -7, 9])[0].get('S')==4 #order of pairs matters
assert get_hidden_sector([7, -8, -10, 11, -12, 13, 14, -15])[0].get('S')==1 #double intersection pair
assert get_hidden_sector([4,-12])[0].get('S')==8

Valentina

In [222]:
import pandas as pd
import os
import numpy as np
import itertools
import collections



def get_hidden_sector(l):
    
    l_new = np.unique(l,return_counts = True)
    com = list(itertools.combinations_with_replacement(l,2))
    suma = set([abs(sum(i)) for i in com])
    
    final = []

#------------------------------------------------------------------------------------------------------------------

    for s in suma:
        
        cond = [i for i in com if abs(sum(i))==s]
        
        aplanar = [item for sublist in cond for item in sublist]
        resta = set(l).difference(set(aplanar))
        
        if not resta:
            if len(l)%2 == 0:
                m = int(len(l)/2)
                a = np.array(list(itertools.combinations(cond,m)))
                com_cond = [np.sort(i.flatten()) for i in a]

                if len(com_cond) != 0:
                    result = np.unique(a[(com_cond == np.sort(np.array(l))).all(axis =1)], axis = 0).tolist()
                    if not result:
                        final.append([])
                    else:
                        final.append({'S': s, 'ψ':[tuple(x) for x in result[0]]})
                else:
                    continue

            else:
                m = int(len(l)/2) + 1
                a = np.array(list(itertools.combinations(cond,m)))
                aux = np.unique(np.array(cond)[np.array([len(np.unique(i)) == 1 for i in cond])])
                sself = [item for item, count in collections.Counter(l).items() if count == 1 and item in aux]

                com_cond = [np.unique(i,return_counts=True) for i in a]

                i = 0
                index = []

                for l2,l2_cuentas in com_cond:
                    if len(l2_cuentas) == len(l_new[1]):
                        if (np.isin(l_new[0][~(l_new[1] == l2_cuentas)],sself)).all():
                            if (l2_cuentas[~(l_new[1] == l2_cuentas)] == l_new[1][~(l_new[1] == l2_cuentas)]+1).all() and np.array([len(np.unique(j))==1 for j in a[i]]).any():
                                index.append(i)

                    i+=1

                if not index:
                    final.append([]) 
                else: 
                    result = np.unique(a[np.array(index)],axis=0).tolist()
                    final.append({'S': s, 'ψ':[tuple(x) for x in result[0]]})
                
    return [x for x in final if x]

assert get_hidden_sector([1,2,-3,4,5])[0].get('S')==6
assert get_hidden_sector([1,2,-3,-3,4,5])[0].get('S')==6
assert get_hidden_sector([1,1,2,-3,-3,4,5,5])[0].get('S')==6
#aψ1ψ1+bψ1ψ2+cψ1,ψ3 → https://www.wolframalpha.com/input?i=Rank%20{{a,b,c},{b,0,0},{c,0,0}}
assert get_hidden_sector([1,2,-3,4,5,9,9])==[]
get_hidden_sector([1,1,2,-3,4,5])
#aψ1ψ2+bψ1ψ3 → https://www.wolframalpha.com/input?i=rank+{{0,a,b},{a,0,0},{b,0,0}}
assert get_hidden_sector([1,1,2,-3,4,5])==[]
assert get_hidden_sector([1,1,2,-3,-3,4,5])==[]
assert get_hidden_sector([1,2,-3,4,5,8])==[]
assert get_hidden_sector([1,2,-3,4,5,8,8])==[]
assert get_hidden_sector([])==[]
assert get_hidden_sector([1, 1, 1, 1, 1, -2, -2, -2, -2, 3])==[]
assert get_hidden_sector([1, 2, 2, 2, -3, -5, -6, 7])[0].get('S')==4
assert get_hidden_sector( [1, 2, 2, 4, -5, -5, -7, 8] )[0].get('S')==3
# Ana test
assert get_hidden_sector([2, -3, -4, 5, -6, 7, 7, -8])[0].get('S')==1
assert get_hidden_sector([3, 5, -8, 9, -10, -14, 15])==[] # Dirac triplet [-10,5,15] (s=5)
assert get_hidden_sector([3, 5, -8, 9, -10, -14, 15, 20,-30, 35])==[]
assert get_hidden_sector([1, 3, 3, 3, -5, -7, -7, 9])[0].get('S')==4 #order of pairs matters
assert get_hidden_sector([7, -8, -10, 11, -12, 13, 14, -15])[0].get('S')==1 #double intersection pair
assert get_hidden_sector([4,-12])[0].get('S')==8

Nicole

In [236]:
import numpy as np
import pandas as pd


# definición de función


def get_hidden_sector(qin):
    """
    Obtiene la masa y los pares que generan masa a las
    soluciones quirales del grupo de simetria U1
    INPUT: q1 - array
    OUTPUT: dict {'S': int, 'Psi': array}
    """
    q = np.array(qin)
    M = np.abs(np.add.outer(q, q))
    sumas = np.unique(M)
    s_ = []
    for suma in sumas:
        ub_s = np.asarray(M == suma).nonzero()

        # garantiza al menos una pareja con ese entero
        if np.unique(ub_s[0]).shape[0] == M.shape[0]:
            s_.append(suma)

    # recorre todas las sumas posibles
    if len(s_) != 0:
        # armar pares
        for s in s_:
            tmp_M = np.asarray(np.triu(M) == s).nonzero()
            pairs = np.array(
                [[q[tmp_M[0][i]], q[tmp_M[1][i]]]
                    for i in range(0, tmp_M[0].shape[0])]
            )

            # agregar condiciones sobre los pares
            if pairs.shape[0] == q.shape[0] and np.sort(pairs.flatten()) == np.sort(q):
                hidden = {"S": s, "ψ": pairs}
                pass
            else:
                # casos no obvios
                # n_pairs = round(q.shape[0] / 2, 0)
                N = q.shape[0]
                q_ = q.copy()

                eq_pairs = [p for p in pairs if p[0] == p[1]]
                dif_pairs = [p for p in pairs if p[0] != p[1]]

                true_pairs = []

                for pair in dif_pairs:
                    # para pares que tomen los elementos
                    if (
                        np.where(q_ == pair[0])[0].shape[0] != 0
                        and np.where(q_ == pair[1])[0].shape[0] != 0
                    ):
                        q_[np.where(q_ == pair[0])[0][0]] = 0
                        q_[np.where(q_ == pair[1])[0][0]] = 0
                        true_pairs.append(pair)
                        N -= 2

                for pair in eq_pairs:
                    # si está una vez
                    if np.where(q_ == pair[0])[0].shape[0] == 1:
                        q_[np.where(q_ == pair[0])[0][0]] = 0
                        true_pairs.append(pair)
                        N -= 1
                    elif np.where(q_ == pair[0])[0].shape[0] > 1:
                        q_[np.where(q_ == pair[0])[0][0]] = 0
                        q_[np.where(q_ == pair[1])[0][0]] = 0
                        true_pairs.append(pair)
                        N -= 2

                if N == 0:
                    hidden = {"S": s, "ψ": true_pairs}
                else:
                    hidden = []
    else:
        hidden = []

    return hidden


# pruebas del hidden sector
assert get_hidden_sector([1, 2, -3, 4, 5]).get("S") == 6
assert get_hidden_sector([1, 2, -3, -3, 4, 5]).get("S") == 6
assert get_hidden_sector([1, 1, 2, -3, -3, 4, 5, 5]).get("S") == 6
assert get_hidden_sector([1, 2, -3, 4, 5, 9, 9]) == []
assert get_hidden_sector([1, 1, 2, -3, 4, 5]) == []
assert get_hidden_sector([1, 1, 2, -3, 4, 5]) == []
assert get_hidden_sector([1, 1, 2, -3, -3, 4, 5]) == []
assert get_hidden_sector([1, 2, -3, 4, 5, 8]) == []
assert get_hidden_sector([1, 2, -3, 4, 5, 8, 8]) == []
assert get_hidden_sector([]) == []
assert get_hidden_sector([1, 1, 1, 1, 1, -2, -2, -2, -2, 3]) == []
assert get_hidden_sector([1, 2, 2, 2, -3, -5, -6, 7]).get("S") == 4
assert get_hidden_sector([1, 2, 2, 4, -5, -5, -7, 8]).get("S") == 3
# Ana test
assert get_hidden_sector([2, -3, -4, 5, -6, 7, 7, -8]).get("S") == 1
assert (
    get_hidden_sector([3, 5, -8, 9, -10, -14, 15]) == []
)  # Dirac triplet [-10,5,15] (s=5)
assert get_hidden_sector([3, 5, -8, 9, -10, -14, 15, 20, -30, 35]) == []

/tmp/ipykernel_352761/402349848.py:37: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if pairs.shape[0] == q.shape[0] and np.sort(pairs.flatten()) == np.sort(q):


In [225]:
%%time
df['hidden']=df['solution'].apply(get_hidden_sector)

/tmp/ipykernel_352761/402349848.py:37: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if pairs.shape[0] == q.shape[0] and np.sort(pairs.flatten()) == np.sort(q):


CPU times: user 1min 45s, sys: 200 ms, total: 1min 45s
Wall time: 1min 45s


In [226]:
hs2=df[df['hidden'].apply(len)>0].reset_index(drop=True)#.iloc[0].to_dict()
hs2.shape

(9029, 7)

In [227]:
hs.shape

(9466, 8)

In [243]:
hs2['hidden']=hs2['hidden'].apply(lambda d: [d])

In [249]:
get_hidden_sector([1, 3, 4, 6, -7, -8, -13, 14])

[{'S': 7, 'ψ': [(3, 4), (6, -13), (1, 6), (-7, 14), (1, -8)]}]

In [247]:
get_hidden_sector([1, 3, 4, 6, -7, -8, -13, 14]) #7 [3,4],[6,-13],[1,6],[-7,14],[1,-8]

[]

In [237]:
get_hidden_sector([1, 1, 2, 3, -4, -4, -5, 6])

/tmp/ipykernel_352761/402349848.py:37: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if pairs.shape[0] == q.shape[0] and np.sort(pairs.flatten()) == np.sort(q):


{'S': 2,
 'ψ': [array([ 2, -4]), array([ 3, -5]), array([-4,  6]), array([1, 1])]}

In [244]:
hs['ss']  = hs['solution'].apply(str)
hs2['ss'] = hs2['solution'].apply(str)

In [192]:
pd.set_option('display.max_colwidth',200)

In [232]:
tmp=hs2[['ss','hidden']].merge(hs[['ss','hidden']],on='ss',how='left',suffixes=('_1','_2'))

In [233]:
tmp[tmp['hidden_2'].apply(lambda x: False if isinstance(x,list) else True )].shape #167

(0, 3)

In [189]:
print( tmp[tmp['hidden_2'].apply(lambda x: False if isinstance(x,list) else True )][:50][["ss",'hidden_1']
].to_json(orient='records',lines=True).replace('\\u03c8','psi').replace('_1','').replace('ss','solution'))

In [245]:
tmp=hs[['ss','hidden']].merge(hs2[['ss','hidden']],on='ss',how='left',suffixes=('_1','_2'))

In [246]:
print( tmp[tmp['hidden_2'].apply(lambda x: False if isinstance(x,list) else True )][:50][["ss",'hidden_1']
].to_json(orient='records',lines=True).replace('\\u03c8','psi').replace('_1','').replace('ss','solution'))

{"solution":"[1, 3, 4, 6, -7, -8, -13, 14]","hidden":[{"S":7,"psi":[[3,4],[6,-13],[1,6],[-7,14],[1,-8]]}]}
{"solution":"[1, -4, -5, 7, -10, 13, 14, -16]","hidden":[{"S":9,"psi":[[-4,-5],[-4,13],[1,-10],[-5,14],[7,-16]]}]}
{"solution":"[2, 4, 5, 7, -8, -13, -14, 17]","hidden":[{"S":9,"psi":[[5,-14],[4,-13],[4,5],[2,7],[-8,17]]}]}
{"solution":"[2, -5, -7, 9, -13, 16, 18, -20]","hidden":[{"S":11,"psi":[[2,-13],[9,-20],[-5,16],[2,9],[-7,18]]}]}
{"solution":"[3, 4, 7, 8, -10, -15, -18, 21]","hidden":[{"S":11,"psi":[[-10,21],[7,-18],[4,7],[3,8],[4,-15]]}]}
{"solution":"[1, -3, -9, 10, -12, 14, 20, -21]","hidden":[{"S":11,"psi":[[10,-21],[1,-12],[-9,20],[-3,14],[1,10]]}]}
{"solution":"[4, 5, 6, 7, -10, -16, -17, 21]","hidden":[{"S":11,"psi":[[5,6],[-10,21],[6,-17],[5,-16],[4,7]]}]}
{"solution":"[1, -7, -7, 11, -13, 19, 19, -23]","hidden":[{"S":12,"psi":[[11,-23],[1,-13],[1,11],[-7,19]]}]}
{"solution":"[5, 5, 7, 7, -11, -17, -19, 23]","hidden":[{"S":12,"psi":[[5,7],[-11,23],[7,-19],[5,-17]]}]}

In [212]:
hs['n'].unique()

array([ 8, 12])

In [ ]:
[1, 3, 4, 6, -7, -8, -13, 14] → [[3,4],[6,-13],[1,6],[-7,14],[1,-8]]
#Remove [1,6] because is already in the pairs list
[1, 3, 4, 6, -7, -8, -13, 14] → [[3,4],[6,-13],[-7,14],[1,-8]]
[1, 6, -7, -8, -13, 14] → [[6,-13],[-7,14],[1,-8]]
[1, -7, -8, 14] → [[-7,14],[1,-8]]
[1, -8] → [[1,-8]]
[] → []

In [181]:
get_hidden_sector(   [1, 3, 4, 6, -7, -8, -13, 14] )

[{'S': 7, 'ψ': [(3, 4), (6, -13), (1, 6), (-7, 14), (1, -8)]}]

## Ask question to the solutions object

* Test the solution
* Which is the minimum hidden sector and with the least charge in absolute value
* Check for multigenerational solution
* Check for unconditional stability
* Solutions with active symmetry
* How many solutions with n=12 and zmax<=20
* Check for Majorana and Dirac fermions

¿Cuales `n` funcionaron? 

In [133]:
hs['n'].unique()

array([ 8, 12])

In [135]:
pd.set_option('display.max_colwidth',200)

In [136]:
# What is the minimum n?
hs['n'].min()

8

In [144]:
# What is the minimum z_max?
hs['zmax'].min()

6

In [145]:
hs[hs['zmax']==hs['zmax'].min()]

,l,k,solution,gcd,n,zmax,hidden
0,"[-1, -5, -3]","[-6, -4, -7]","[1, 1, 2, 3, -4, -4, -5, 6]",1,8,6,"[{'S': 2, 'ψ': [(3, -5), (2, -4), (-4, 6), (1, 1)]}]"
1773,"[-1, 0, -1, 0, 1]","[-3, -2, -3, -2, 2]","[1, -2, -2, -2, 3, 3, 4, 4, -5, -5, -5, 6]",1,12,6,"[{'S': 1, 'ψ': [(-5, 6), (-2, 3), (4, -5), (1, -2)]}, {'S': 7, 'ψ': [(3, 4), (1, 6), (-2, -5)]}]"
1774,"[2, 1, -1, 3, -1]","[-3, -1, 0, -2, -3]","[1, 1, 1, 1, -2, -2, -2, 3, 3, -5, -5, 6]",1,12,6,"[{'S': 4, 'ψ': [(-2, -2), (1, -5), (-2, 6), (1, 3)]}]"


In [146]:
# How many solutions have several dark sectors?
hs[hs['hidden'].apply(len)>1].shape

(1271, 7)

In [147]:
hs[hs['hidden'].apply(len)>2].shape

(0, 7)

In [151]:
hs['hidden'].str[0].str['ψ']

0                                           [(3, -5), (2, -4), (-4, 6), (1, 1)]
1                                           [(2, 2), (-3, 7), (1, -5), (2, -6)]
2                                                    [(4, 5), (1, 8), (-2, -7)]
3                                           [(4, -7), (-5, 8), (2, -5), (1, 2)]
4                                 [(-6, 7), (7, -8), (5, -6), (2, -3), (-4, 5)]
                                         ...                                   
9461                         [(4, -19), (3, -18), (-13, 28), (17, -32), (6, 9)]
9462    [(-9, 27), (-6, -12), (1, -19), (4, 14), (-9, -9), (14, -32), (-8, 26)]
9463              [(-4, -5), (18, -27), (3, 6), (-5, 14), (-23, 32), (14, -23)]
9464                      [(12, -27), (4, -19), (-13, 28), (17, -32), (-9, 24)]
9465                [(6, 18), (5, 19), (-4, -20), (3, -27), (2, -26), (-8, 32)]
Name: hidden, Length: 9466, dtype: object

In [157]:
# What is the minimum number of DM particles?
hs['hidden'].str[0].str['ψ'].apply(len).min()

3

In [156]:
# What is the minimum number of DM particles?
hs['hidden'].str[-1].str['ψ'].apply(len).min()

3

In [161]:
hs.loc[[1773]]['hidden'].str[-1]

1773    {'S': 7, 'ψ': [(3, 4), (1, 6), (-2, -5)]}
Name: hidden, dtype: object

In [159]:
hs.loc[[1773]]['hidden'].str[0]

,l,k,solution,gcd,n,zmax,hidden
1773,"[-1, 0, -1, 0, 1]","[-3, -2, -3, -2, 2]","[1, -2, -2, -2, 3, 3, 4, 4, -5, -5, -5, 6]",1,12,6,"[{'S': 1, 'ψ': [(-5, 6), (-2, 3), (4, -5), (1, -2)]}, {'S': 7, 'ψ': [(3, 4), (1, 6), (-2, -5)]}]"


In [166]:
True or False

True

In [169]:
hs[(hs['hidden'].str[0].str['ψ'].apply(len)==3)].shape

(221, 7)

In [170]:
hs[(hs['hidden'].str[0].str['ψ'].apply(len)==3) | 
   (hs['hidden'].str[-1].str['ψ'].apply(len)==3)]

,l,k,solution,gcd,n,zmax,hidden
2,"[-1, 0, 1]","[-4, -2, 2]","[1, -2, -2, 4, 5, -7, -7, 8]",1,8,8,"[{'S': 9, 'ψ': [(4, 5), (1, 8), (-2, -7)]}]"
7,"[2, 0, -1]","[-8, -5, 9]","[2, -3, -3, 5, 6, -8, -8, 9]",1,8,9,"[{'S': 11, 'ψ': [(5, 6), (-3, -8), (2, 9)]}]"
9,"[0, -3, -1]","[-3, -5, -6, -4]","[3, -4, -6, -6, 7, 7, 8, -9]",1,8,9,"[{'S': 1, 'ψ': [(8, -9), (-6, 7), (3, -4)]}]"
11,"[-1, 0, -1]","[-2, 1, -1]","[2, -5, -5, -5, 7, 8, 8, -10]",1,8,10,"[{'S': 3, 'ψ': [(7, -10), (-5, 8), (2, -5)]}]"
14,"[-3, -5, 1]","[-6, -4, -5]","[3, -4, -4, 6, 7, -9, -9, 10]",1,8,10,"[{'S': 13, 'ψ': [(3, 10), (-4, -9), (6, 7)]}]"
...,...,...,...,...,...,...,...
7956,"[1, -2, 1, 0, 1]","[-2, -1, -2, -1, -2]","[1, 1, -2, -2, -16, -16, -16, 17, 17, 17, 28, -29]",1,12,29,"[{'S': 1, 'ψ': [(-16, 17), (28, -29), (1, -2)]}]"
7959,"[-2, 1, -2, 0, 2]","[-2, 1, -2, 1, -1]","[4, -8, -8, -8, 11, 11, 22, 22, -25, -25, -25, 29]",3,12,29,"[{'S': 33, 'ψ': [(4, 29), (11, 22), (-8, -25)]}]"
7991,"[-1, 0, -2, 0, -2]","[-3, 1, -3, 1, -3]","[7, 7, -9, -9, -9, 17, 19, -27, -27, -27, 29, 29]",4,12,29,"[{'S': 36, 'ψ': [(7, 29), (-9, -27), (17, 19)]}]"
7997,"[-2, 0, -2, 0, -2]","[-3, 2, -3, 3, -3]","[16, 16, -19, -19, -19, -21, -21, 24, 24, 24, 24, -29]",8,12,29,"[{'S': 5, 'ψ': [(-19, 24), (24, -29), (16, -21)]}]"


In [171]:
# What is the maximum number of DM particles?
hs['hidden'].str[0].str['ψ'].apply(len).max()

9

In [172]:
# What is the maximum number of DM particles?
hs['hidden'].str[-1].str['ψ'].apply(len).max()

9

In [176]:
hs[(hs['hidden'].str[0].str['ψ'].apply(len)==9)][:1]

,l,k,solution,gcd,n,zmax,hidden
2019,"[-1, 0, -1, -2, -1]","[-3, -2, 2, 1, -1]","[1, 1, 2, 3, -5, -6, -6, -7, 9, 10, 11, -13]",2,12,13,"[{'S': 4, 'ψ': [(9, -13), (-5, 9), (-6, 10), (2, 2), (1, 3), (-7, 11), (3, -7), (1, -5), (2, -6)]}, {'S': 12, 'ψ': [(-5, -7), (3, 9), (-6, -6), (1, -13), (1, 11), (2, 10)]}]"


### Same questions for SM solutions

In [23]:
df3=df[df['solution'].apply(lambda l: 3 in [l.count(x) for x in l])].reset_index(drop=True)

In [24]:
def get_dark(l,idx=0):
    #TODO,
    xxx=list(set([x for x in l if l.count(x)==3]))
    if len(xxx)>=idx+1:
        xx=xxx[idx]
        return [x for x in l if x!=xx]
    else:
        return []
assert get_dark([1,1,2,2,2,3] )==[1, 1, 3]
assert get_dark([2,2,2,2,-5,-5,-5,7])==[2, 2, 2, 2, 7]

In [25]:
df3['hidden']=df3['solution'].apply(get_dark).apply(get_hidden_sector)

In [26]:
hs3=df3[df3['hidden'].apply(len)>0].reset_index(drop=True)
hs3.shape

(192, 7)

In [27]:
# idxmax==1
df3['hidden']=df3['solution'].apply(lambda l: get_dark(l,idx=1)).apply(get_hidden_sector)
hs3=pd.concat((hs3,df3[df3['hidden'].apply(len)>0])).reset_index(drop=True)
hs3.shape

(199, 7)

In [28]:
# idxmax==1
df3['hidden']=df3['solution'].apply(lambda l: get_dark(l,idx=2)).apply(get_hidden_sector)
hs3=pd.concat((hs3,df3[df3['hidden'].apply(len)>0])).reset_index(drop=True)
hs3.shape

(199, 7)

In [29]:
hs3['zmax'].min()

9

In [30]:
hs3[:1]

,l,k,solution,gcd,n,zmax,hidden
0,"[-3, -1, 2]","[1, 3, -3]","[3, 3, 3, 5, -7, -9, -11, 13]",12,8,13,"[{'S': 18, 'ψ': [(-7, -11), (-9, -9), (5, 13)]}]"


In [31]:
hs3[hs3['hidden'].apply(len)>1].shape

(0, 7)

In [32]:
hs3[hs3['hidden'].apply(len)>2].shape

(0, 7)

In [33]:
hs3[hs3['hidden'].apply(len)>3].shape

(0, 7)

In [34]:
hs3['hidden'].str[0].str['ψ'].apply(len).min()

2

In [35]:
hs3[hs3['hidden'].str[0].str['ψ'].apply(len)==hs3['hidden'].str[0].str['ψ'].apply(len).min()]

,l,k,solution,gcd,n,zmax,hidden
3,"[-2, 0, 2]","[-2, 1, 0, -2]","[1, 1, -4, -5, 9, 9, 9, -10, -10]",4,9,10,"[{'S': 9, 'ψ': [(1, -10), (-4, -5)]}]"
4,"[1, 0, -1]","[-3, 3, 0, -3]","[1, 1, -3, -8, 11, 11, 11, -12, -12]",9,9,12,"[{'S': 11, 'ψ': [(1, -12), (-3, -8)]}]"
6,"[3, 0, -3]","[-2, 2, 0, -2]","[4, -5, -5, -6, -6, 11, 11, 11, -15]",8,9,15,"[{'S': 11, 'ψ': [(-5, -6), (4, -15)]}]"
13,"[2, 0, -2]","[2, -2, 0, -3]","[2, 2, -3, 15, 15, -17, -17, -17, 20]",4,9,20,"[{'S': 17, 'ψ': [(-3, 20), (2, 15)]}]"
16,"[2, 0, -2]","[3, -3, 0, 3]","[3, 3, -4, 14, 14, -17, -17, -17, 21]",9,9,21,"[{'S': 17, 'ψ': [(-4, 21), (3, 14)]}]"
17,"[-3, 0, 1]","[-3, 3, 0, -3]","[2, 2, -7, -12, 19, 19, 19, -21, -21]",9,9,21,"[{'S': 19, 'ψ': [(-7, -12), (2, -21)]}]"
22,"[3, 0, 1]","[3, -3, 0, 3]","[4, 4, -5, 15, 15, -19, -19, -19, 24]",9,9,24,"[{'S': 19, 'ψ': [(4, 15), (-5, 24)]}]"


### Extend the data scheme with number of generations

In [177]:
def get_Dirac_and_Majorana_generations(l,h):
    g=[]
    dgmax=0
    mgmax=0
    for idx in range(len(h)):
        for x,y in h[idx].get('ψ'):
            if x!=y and l.count(x)>dgmax and l.count(y)==l.count(x):
                    dgmax=l.count(x)
            if x==y and l.count(x)>mgmax:
                    mgmax=l.count(x)
        g.append({'DG':dgmax,'MG':mgmax})
    return g
l=[2, 2, -3, 15, 15, 20]
h=[{'S': 17, 'ψ': [(2, 15), (20, -3)]}]
assert get_Dirac_and_Majorana_generations(l,h)==[{'DG': 2, 'MG': 0}]

In [181]:
hs[:3].apply(lambda d: d['l']+d['k'],axis='columns')

0    [-1, -5, -3, -6, -4, -7]
1       [-1, 2, -2, -7, 4, 0]
2       [-1, 0, 1, -4, -2, 2]
dtype: object

In [182]:
hs['Generations']=hs.apply(lambda row: get_Dirac_and_Majorana_generations(row['solution'],row['hidden']),axis='columns')

In [38]:
hs3['Generations']=hs3.apply(lambda row: get_Dirac_and_Majorana_generations(row['solution'],row['hidden']),axis='columns')

In [187]:
hs['Gmax']=hs['Generations'].str[0].apply(lambda d: sum(d.values()))

In [188]:
hs=hs.sort_values(['Gmax'],ascending=False).reset_index(drop=True)

In [189]:
hs[hs['n']==8]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
453,"[1, 0, -2]","[-3, -1, 1]","[3, 3, 3, 5, -7, -9, -11, 13]",2,8,13,"[{'S': 6, 'ψ': [(5, -11), (3, -9), (3, 3), (-7, 13)]}]","[{'DG': 1, 'MG': 3}]",4
454,"[-1, 2, -2]","[-7, 4, 0]","[1, 2, 2, 2, -3, -5, -6, 7]",1,8,7,"[{'S': 4, 'ψ': [(2, 2), (-3, 7), (1, -5), (2, -6)]}]","[{'DG': 1, 'MG': 3}]",4
455,"[0, -1, -2]","[-1, -6, -5, -6]","[1, -6, -6, -6, 7, 11, 18, -19]",2,8,19,"[{'S': 12, 'ψ': [(-6, -6), (-6, 18), (1, 11), (7, -19)]}]","[{'DG': 1, 'MG': 3}]",4
456,"[-3, -4, -3]","[-5, 3, -7]","[3, 5, 5, 7, -9, -15, -15, 19]",2,8,19,"[{'S': 10, 'ψ': [(3, 7), (5, -15), (-9, 19), (5, 5)]}]","[{'DG': 2, 'MG': 2}]",4
457,"[3, 0, -5]","[-1, 1, -1]","[4, 4, 4, 11, -12, -13, -19, 21]",8,8,21,"[{'S': 8, 'ψ': [(11, -19), (-13, 21), (4, 4), (4, -12)]}]","[{'DG': 1, 'MG': 3}]",4
...,...,...,...,...,...,...,...,...,...
9029,"[-3, -2, 1]","[-7, -13, -1]","[3, 3, 5, 7, -9, -11, -15, 17]",2,8,17,"[{'S': 8, 'ψ': [(-9, 17), (7, -15), (3, 5), (3, -11)]}]","[{'DG': 1, 'MG': 0}]",1
9030,"[3, 2, -2]","[-9, -14, -4]","[1, -2, -3, 6, 9, -12, -16, 17]",1,8,17,"[{'S': 15, 'ψ': [(-3, -12), (-2, 17), (1, -16), (6, 9)]}]","[{'DG': 1, 'MG': 0}]",1
9031,"[0, -3, -4]","[-1, -5, -3, 2]","[1, 1, 2, -6, -10, 14, 15, -17]",1,8,17,"[{'S': 16, 'ψ': [(2, 14), (1, 15), (-6, -10), (1, -17)]}]","[{'DG': 1, 'MG': 0}]",1
9032,"[0, -2, -4]","[-3, -5, -6, -4]","[1, -6, -7, -9, 12, 12, 14, -17]",2,8,17,"[{'S': 5, 'ψ': [(1, -6), (-9, 14), (12, -17), (-7, 12)]}]","[{'DG': 1, 'MG': 0}]",1


In [42]:
hs[hs['n']==8][:6]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
0,"[1, -2, 2]","[2, -1, 0]","[1, 2, 2, 2, -3, -5, -6, 7]",1,8,7,"[{'S': 4, 'ψ': [(2, 2), (-3, 7), (1, -5), (2, -6)]}]","[{'DG': 1, 'MG': 3}]",-4
1,"[-3, -1, 2]","[1, 3, -3]","[3, 3, 3, 5, -7, -9, -11, 13]",12,8,13,"[{'S': 6, 'ψ': [(5, -11), (3, -9), (3, 3), (-7, 13)]}]","[{'DG': 1, 'MG': 3}]",-4
2,"[-3, 0, -1]","[1, 3, 1]","[3, 5, 5, 7, -9, -15, -15, 19]",4,8,19,"[{'S': 10, 'ψ': [(3, 7), (5, -15), (-9, 19), (5, 5)]}]","[{'DG': 2, 'MG': 2}]",-4
3,"[-1, 0, -5]","[3, 1, -1]","[1, -6, -6, -6, 7, 11, 18, -19]",8,8,19,"[{'S': 12, 'ψ': [(-6, -6), (-6, 18), (1, 11), (7, -19)]}]","[{'DG': 1, 'MG': 3}]",-4
4,"[3, 0, -5]","[-1, 1, -1]","[4, 4, 4, 11, -12, -13, -19, 21]",8,8,21,"[{'S': 8, 'ψ': [(11, -19), (-13, 21), (4, 4), (4, -12)]}]","[{'DG': 1, 'MG': 3}]",-4
5,"[1, -1, 1]","[1, 0, -2]","[1, 1, 2, 3, -4, -4, -5, 6]",2,8,6,"[{'S': 2, 'ψ': [(3, -5), (2, -4), (-4, 6), (1, 1)]}]","[{'DG': 1, 'MG': 2}]",-3


In [43]:
hs[hs['n']==9][:3]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax


In [44]:
hs[hs['n']==11][:4]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax


In [45]:
hs[hs['n']==12][:4]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
1388,"[2, 0, -2, -4, -2]","[2, -2, 2, -3, -1]","[2, 2, 2, 2, 2, 2, 5, -6, -6, -7, -9, 11]",8,12,11,"[{'S': 4, 'ψ': [(2, 2), (-7, 11), (5, -9), (2, -6)]}]","[{'DG': 1, 'MG': 6}]",-7
1389,"[3, 5, 3, 0, 3]","[3, -2, 4, -3, 3]","[3, 3, 3, 3, 3, 3, 4, -8, -9, -9, -10, 14]",12,12,14,"[{'S': 6, 'ψ': [(4, -10), (3, -9), (3, 3), (-8, 14)]}]","[{'DG': 1, 'MG': 6}]",-7
1390,"[-2, 3, -7, -3, -7]","[-2, -6, -2, 2, 6]","[1, 1, 7, 7, 7, 7, 7, -11, -15, -15, -21, 25]",16,12,25,"[{'S': 14, 'ψ': [(1, -15), (7, 7), (-11, 25), (7, -21)]}]","[{'DG': 2, 'MG': 5}]",-7
1391,"[-7, 10, 5, -2, 6]","[-7, 1, -9, 3, -9]","[5, 5, 7, 7, 7, 7, 7, -15, -19, -19, -21, 29]",220,12,29,"[{'S': 14, 'ψ': [(-15, 29), (7, 7), (7, -21), (5, -19)]}]","[{'DG': 2, 'MG': 5}]",-7


In [46]:
hs3['Gmax']=-hs3['Generations'].str[0].apply(lambda d: sum(d.values()))
hs3=hs3.sort_values(['n','Gmax']).reset_index(drop=True)

In [47]:
hs3[hs3['n']==8][:1]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
0,"[-3, -1, 2]","[1, 3, -3]","[3, 3, 3, 5, -7, -9, -11, 13]",12,8,13,"[{'S': 18, 'ψ': [(-7, -11), (-9, -9), (5, 13)]}]","[{'DG': 1, 'MG': 1}]",-2


In [48]:
hs3[hs3['n']==9][:4]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
2,"[-2, -3, 0]","[1, 2, 3, 2]","[5, 6, 6, 6, -8, -9, -9, -10, 13]",4,9,13,"[{'S': 18, 'ψ': [(-9, -9), (-8, -10), (5, 13)]}]","[{'DG': 1, 'MG': 2}]",-3
3,"[-4, -2, 1]","[2, -4, 4, -2]","[1, -2, 6, 6, 6, -9, -9, -16, 17]",16,9,17,"[{'S': 18, 'ψ': [(-9, -9), (1, 17), (-2, -16)]}]","[{'DG': 1, 'MG': 2}]",-3
4,"[-2, 0, 2]","[-2, 1, 0, -2]","[1, 1, -4, -5, 9, 9, 9, -10, -10]",4,9,10,"[{'S': 9, 'ψ': [(1, -10), (-4, -5)]}]","[{'DG': 2, 'MG': 0}]",-2
5,"[1, 0, -1]","[-3, 3, 0, -3]","[1, 1, -3, -8, 11, 11, 11, -12, -12]",9,9,12,"[{'S': 11, 'ψ': [(1, -12), (-3, -8)]}]","[{'DG': 2, 'MG': 0}]",-2


In [49]:
hs3[hs3['n']==10][:5]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
37,"[3, -3, 0, 6]","[3, -2, -3, -4]","[3, 3, 3, -4, 9, 9, -10, -14, -27, 28]",18,10,28,"[{'S': 18, 'ψ': [(-10, 28), (-4, -14), (9, -27), (9, 9)]}]","[{'DG': 1, 'MG': 2}]",-3
38,"[3, -3, 5, -2]","[3, 5, -3, 1]","[1, -3, -3, -3, -9, -9, 11, 17, 27, -29]",10,10,29,"[{'S': 18, 'ψ': [(-9, 27), (11, -29), (1, 17), (-9, -9)]}]","[{'DG': 1, 'MG': 2}]",-3
39,"[-3, -1, -3, -1]","[-3, 0, 2, -1]","[3, 3, 3, 4, -6, -7, 9, -11, -12, 14]",6,10,14,"[{'S': 18, 'ψ': [(-6, -12), (4, 14), (-7, -11), (9, 9)]}]","[{'DG': 1, 'MG': 1}]",-2
40,"[-1, 0, 2, -1]","[3, -3, 3, -1]","[1, -5, -7, 9, -11, -13, 15, 15, 15, -19]",4,10,19,"[{'S': 18, 'ψ': [(-5, -13), (1, -19), (-7, -11), (9, 9)]}]","[{'DG': 1, 'MG': 1}]",-2
41,"[-2, 1, 0, 3]","[3, 0, -3, 1]","[1, -2, 3, 3, 3, -6, 9, -12, -19, 20]",6,10,20,"[{'S': 18, 'ψ': [(-2, 20), (-6, -12), (1, -19), (9, 9)]}]","[{'DG': 1, 'MG': 1}]",-2


In [50]:
hs3[hs3['n']==11][:2]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
43,"[-4, 0, 2, 0]","[-4, 4, -4, 3, -3]","[4, 5, 8, -9, -9, 10, -12, -12, -12, 13, 14]",64,11,14,"[{'S': 18, 'ψ': [(5, 13), (4, 14), (-9, -9), (8, 10)]}]","[{'DG': 1, 'MG': 2}]",-3
44,"[-8, -2, 6, 1]","[5, -5, -6, -2, -9]","[1, -5, 9, 9, -11, 17, 23, -24, -24, -24, 29]",73,11,29,"[{'S': 18, 'ψ': [(-11, 29), (-5, 23), (1, 17), (9, 9)]}]","[{'DG': 1, 'MG': 2}]",-3


In [51]:
hs3[hs3['n']==12][:5]

,l,k,solution,gcd,n,zmax,hidden,Generations,Gmax
107,"[-1, -6, -5, 1, 7]","[-4, -7, -2, 7, 4]","[2, 8, -9, -9, -9, -9, -9, 10, 15, 15, 15, -20]",126,12,20,"[{'S': 18, 'ψ': [(-9, -9), (8, 10), (2, -20)]}]","[{'DG': 1, 'MG': 5}]",-6
108,"[9, 6, 8, 5, 3]","[9, 3, -10, 1, 2]","[4, -8, 9, 9, 9, 9, 9, -15, -15, -15, -22, 26]",324,12,26,"[{'S': 18, 'ψ': [(4, -22), (9, 9), (-8, 26)]}]","[{'DG': 1, 'MG': 5}]",-6
109,"[-3, -9, 0, -3, 3]","[9, -10, -9, -3, 4]","[2, 3, 3, 3, -9, -9, -9, -9, 10, 16, 27, -28]",36,12,28,"[{'S': 18, 'ψ': [(-9, 27), (10, -28), (2, 16), (-9, -9)]}]","[{'DG': 1, 'MG': 4}]",-5
110,"[-1, 3, 2, 3, -1]","[3, -3, -1, -3, -1]","[1, 1, 3, 3, 3, 5, 5, -7, -9, -9, -9, 13]",8,12,13,"[{'S': 6, 'ψ': [(1, 5), (3, 3), (-7, 13), (1, -7)]}]","[{'DG': 2, 'MG': 3}]",-5
111,"[1, 3, 2, 0, -3]","[-3, -2, -1, -2, -3]","[3, 3, 3, 6, 7, -8, -9, -9, -9, -10, 11, 12]",8,12,12,"[{'S': 18, 'ψ': [(6, 12), (-9, -9), (-8, -10), (7, 11)]}]","[{'DG': 1, 'MG': 3}]",-4


In [110]:
import csv as CSV
class read_csv:
    def __init__(self,f):
        self.data=[]
        f=open(f,'r')
        dt=CSV.reader(f)
        for row in dt:
              self.data.append(row)
        f.close()
    def to_json(self):
        return [dict(zip(data[0],d)) for d in self.data[1:]]

In [111]:
csv=read_csv('test.csv')

In [112]:
csv.data

[['A', 'B'], ['1', '2'], ['3', '4']]

In [113]:
csv.to_json()

[{'A': '1', 'B': '2'}, {'A': '3', 'B': '4'}]